# Extraction of timedate when it was published

In [ ]:
#!/usr/bin/env python
import requests
import json
import pandas as pd
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

base = 'https://dataverse.harvard.edu'
rows = 1000
start = 0
page = 1
condition = True # emulate do-while
prueba = requests.Session()
retry = Retry(connect=3, backoff_factor=2)
adapter = HTTPAdapter(max_retries=retry)
prueba.mount('http://', adapter)
prueba.mount('https://', adapter)
scraping_types=pd.DataFrame(columns=['Type'])
scraping_names=pd.DataFrame(columns=['Name'])
scraping_publication=pd.DataFrame(columns=['PublicationDate'])



while (condition):
    url = base + '/api/search?q=*&type=file&show_facets=true&per_page=1000' + "&start=" + str(start)
    data = json.loads(prueba.get(url).text)
    total = data['data']['total_count']
    print("=== Page", page, "===")
    print("start:", start, " total:", total)
    for i in data['data']['items']:
        scraping_types=scraping_types.append(pd.DataFrame([i['type']], columns=['Type']))
        scraping_names=scraping_names.append(pd.DataFrame([i['name']], columns=['Name']))
        scraping_publication=scraping_publication.append(pd.DataFrame([i['published_at']], columns=['PublicationDate']))


    start = start + rows
    page += 1
    condition = start < total/5

=== Page 1 ===
start: 0  total: 490708


# Extraction of file formats

In [ ]:
#!/usr/bin/env python
import requests
import json
import pandas as pd
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

base = 'https://dataverse.harvard.edu'
rows = 10
start = 0
page = 1
condition = True # emulate do-while
prueba = requests.Session()
retry = Retry(connect=3, backoff_factor=2)
adapter = HTTPAdapter(max_retries=retry)
prueba.mount('http://', adapter)
prueba.mount('https://', adapter)
filetypes_count=pd.DataFrame(columns=['count'])
filetypes_name=pd.DataFrame(columns=['name'])

url = base + '/api/search?q=*&type=file&show_facets=true&per_page=1' + "&start=" + str(start)
data = json.loads(prueba.get(url).text)
total = data['data']['total_count']
print(" total:", total)

    
for i in data['data']['facets']:
    for d in i['fileTypeGroupFacet']['labels']:
        for j in d:
            filetypes_count=filetypes_count.append(pd.DataFrame([d[j]], columns=['count']))
            filetypes_name=filetypes_name.append(pd.DataFrame([j], columns=['name']))

final=pd.concat([filetypes_name,filetypes_count],axis=1).reset_index()


print(final)

In [ ]:
final.iloc[:, 1:].to_csv('fileformats.csv',sep=",", encoding="UTF-8")

In [56]:
final.index.name = 'index'
print(final)

       index          name   count
index                             
0          0          Data  114574
1          0       Unknown   71564
2          0      Document   59287
3          0  Tabular Data   47734
4          0          Text   45593
5          0         Image   40544
6          0   Application   23722
7          0           ZIP    5894
8          0          FITS    4995
9          0         Audio    3618
10         0         Video     940
11         0         Shape     871
12         0  Network Data      55
13         0      Chemical       9
14         0        Binary       7
15         0         Model       2


In [37]:
print(scraping_publication)
print(scraping_names)
print(scraping_types)

         PublicationDate
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
0   2011-12-13T00:00:00Z
..                   ...
0   2015-04-09T03:06:40Z
0   2015-04-09T03:06:40Z
0   2015-04-09T03:06:40Z
0   2015-04-09T03:06:40Z
0   2015-04-09T03:06:40Z
0   2015-04-09T03:06:40Z
0   2015-04-09T03:06:40Z
0   2015-04-09T03:06:40Z


In [20]:
final=pd.concat([scraping_publication,scraping_names,scraping_types, scraping_publicationtime],axis=1).reset_index()
final.to_csv('datasearch.csv',sep=",", encoding="UTF-8")

In [19]:
import datetime

scraping_publicationtime=pd.DataFrame(columns=['PublicationTime'])

holi=pd.DataFrame(scraping_publication['PublicationDate'])
holi['PublicationDate'] = pd.to_datetime(holi['PublicationDate'])
for i in holi['PublicationDate']:
    i= i-datetime.timedelta(minutes=i.minute % 10,
                             seconds=i.second,
                             microseconds=i.microsecond)
    i += datetime.timedelta(minutes=5)
    i -= datetime.timedelta(minutes=i.minute % 10,
                         seconds=i.second,
                         microseconds=i.microsecond)
    i=i.strftime("%H:%M")
    scraping_publicationtime=scraping_publicationtime.append(pd.DataFrame([i], columns=['PublicationTime']))
print(scraping_publicationtime)


   PublicationTime
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
..             ...
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0            00:00
0           

In [24]:
scraping_publicationtime=pd.DataFrame(scraping_publicationtime, columns=['PublicationTime'])
scraping=pd.concat([scraping_publicationtime,scraping_types], axis=1)
print(scraping)

     PublicationTime  level_0  index  Type
0              00:00        0      0  file
1              00:00        1      0  file
2              00:00        2      0  file
3              00:00        3      0  file
4              00:00        4      0  file
5              00:00        5      0  file
6              00:00        6      0  file
7              00:00        7      0  file
8              00:00        8      0  file
9              00:00        9      0  file
10             00:00       10      0  file
11             00:00       11      0  file
12             00:00       12      0  file
13             00:00       13      0  file
14             00:00       14      0  file
15             00:00       15      0  file
16             00:00       16      0  file
17             00:00       17      0  file
18             00:00       18      0  file
19             00:00       19      0  file
20             00:00       20      0  file
21             00:00       21      0  file
22         

In [21]:
scraping2=scraping_publicationtime.reset_index().groupby('PublicationTime').count()
print(scraping2)
scraping2.to_csv("data_circular.csv", sep=";", encoding="UTF-8")

                 index
PublicationTime       
00:00            34075
03:00             4154
03:10             6050
03:20             6236
03:30             7003
03:40             7437
03:50             9355
04:00             7560
04:10             6111
18:30               19


In [75]:
import csv
import pandas
with open('datasearchFINAL.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    print(spamreader)
    index=pd.DataFrame(columns=["index"])
    PublicationTime=pd.DataFrame(columns=["PublicationTime"])
    for j in spamreader:
            index=index.append(pd.DataFrame([int(j[1])], columns=["index"]))
            PublicationTime=PublicationTime.append(pd.DataFrame([j[0]], columns=["PublicationTime"]))
    holi=pd.concat([PublicationTime, index],axis=1)
    holi3=holi.groupby('PublicationTime')['index'].sum()
    holi3.to_csv("data_circularFINAL.csv", sep=",", encoding="UTF-8")